In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
target_files = ["1652875851.3497071.csv", "1652875901.3107166.csv", "1652876013.741493.csv", "1652876206.2541456.csv", "1652876485.8123376.csv", "1652959186.4507334.csv",
                "1652959347.972946.csv", "1653042695.4914637.csv", "1653042775.5213027.csv", "1653043202.5073502.csv", "1653043345.3415065.csv", "1653043428.8546412.csv", "1653043549.5187616.csv"]
image_dirs = ['.'.join(x.split(".")[:-1]) for x in target_files]

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_dirs, target_files):
        self.image_dirs = image_dirs
        self.target_files = target_files
        self.images = []
        for image_dir in image_dirs:
            for filename in sorted(os.listdir(os.path.join('dataset', image_dir))):
                if filename.endswith('.jpg'):
                    self.images.append(os.path.join('dataset', image_dir, filename))
        self.targets = pd.DataFrame(columns=['index', 'left', 'right'])
        for target_file in target_files:
            target_path = os.path.join('dataset', target_file)
            if os.path.exists(target_path):
                next_frame = pd.read_csv(target_path)
                self.targets = pd.concat([self.targets, next_frame], axis=0, ignore_index=True)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        target = self.targets.iloc[idx]
        target = torch.tensor([target['left'], target['right']])
        image = transforms.ToTensor()(Image.open(image_path))
        image = transforms.Resize((3, 224, 224))(image)
        
        return image, target

In [5]:
img_dataset = ImageDataset(image_dirs, target_files)

In [6]:
train_dataset, val_dataset = random_split(img_dataset, [0.8, 0.2])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
class DrivingModel(nn.Sequential):
    def __init__(self):
        super(DrivingModel, self).__init__()
        self.input_shape = (1, 3, 224, 224)
        self.append(nn.Conv2d(3, 16, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.Conv2d(16, 32, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        self.append(nn.Conv2d(32, 64, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.Conv2d(64, 128, kernel_size=3, padding=1))
        self.append(nn.ReLU())
        self.append(nn.MaxPool2d(kernel_size=2, stride=2))
        self.append(nn.Flatten())
        self.append(nn.Linear(128 * 56 * 56, 512))
        self.append(nn.ReLU())
        self.append(nn.Linear(512, 64))
        self.append(nn.ReLU())
        self.append(nn.Linear(64, 2))
    
    def forward(self, x):
        x = x.view(-1, 3, 224, 224)
        for layer in self:
            x = layer(x)
        return x

In [ ]:
model = DrivingModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 100
for epoch in range(num_epochs):
    train_loss = 0
    model.train()
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images = images.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss / len(train_loader)}")
    
    val_loss = 0
    model.eval()
    for images, targets in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
        images = images.to(device)
        targets = targets.to(device)
        with torch.no_grad():
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss / len(val_loader)}")

Epoch 1/100 - Training:   0%|          | 0/190 [00:00<?, ?it/s]


ValueError: If size is a sequence, it should have 1 or 2 values